In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

%load_ext nb_black
%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
# import helper.add_features
# import importlib

# importlib.reload(helper.add_features)

<IPython.core.display.Javascript object>

#### 读取包

In [3]:
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import sys
import os
import gc
import random
import pickle
from more_itertools import consecutive_groups
from ipywidgets import interact, fixed, IntSlider
import subprocess


np.set_printoptions(suppress=True)
pd.set_option("display.max_rows", 200)
sys.path = sys.path + ["/home/qlyang/Documents/pacman/"]

from helper.utiltools import eval_df
from helper.pacmanutils import (
    rt_df_filter,
    generate_local_4dirs,
    relative_dir,
    largest_2ndlargest_diff,
    to_game,
    if_get_nearbean,
    assign_category,
    combine_pre_post,
    add_stats,
    add_combine_huntdis,
    generate_simulated_local_4dirs,
    get_marker,
    plot_ghost,
    plot_colors_simple,
)

from helper.add_features import add_dis, add_move_dir
from helper.constant_input import (
    OPPOSITE_DIRS,
    MAP_INFO,
    TURNING_POS,
    LOCS_DF,
    POSSIBLE_DIRS,
    SYMBOLS,
    ARRAY,
)
from helper.analysis import status_index

from ipywidgets import interact, fixed
import ipywidgets as widgets

<IPython.core.display.Javascript object>

#### 读取functions

In [4]:
def set_landcolor(landscape_colors, landscape=True, land=None):
    if landscape:
        sns.set_palette(landscape_colors.values())
    else:
        if land == "straight":
            sns.set_palette(plt.cm.Reds(np.linspace(0.3, 1, 4)))
        if land == "L-shape":
            sns.set_palette(plt.cm.Blues(np.linspace(0.3, 1, 4)))
        if land == "fork":
            sns.set_palette(plt.cm.Greens(np.linspace(0.3, 1, 4)))
        if land == "cross":
            sns.set_palette(plt.cm.Wistia(np.linspace(0.3, 1, 4)))


def extend_df_overlap(df_total, condition, data_type="simulated"):
    if data_type != "simulated":
        df_overlap = pd.read_pickle(
            "../constants/df_overlap_" + "omega" + ".pkl"
        ).append(pd.read_pickle("../constants/df_overlap_" + "patamon" + ".pkl"))
    else:
        df_overlap = pd.read_pickle("../constants/df_overlap_simulated.pkl")
    df_filter = df_total.loc[
        condition,
        ["file", "index", "next_pacman_dir_fill", "pacmanPos", "pacman_dir_fill",],
    ]
    df_overlap = (
        df_overlap.assign(
            local_4dirs_diff=largest_2ndlargest_diff(df_overlap),
            largest_dir=df_overlap.eq(df_overlap.max(1), axis=0)
            .stack()
            .replace({False: np.nan})
            .dropna()
            .reset_index()
            .groupby(["file", "index"])
            .apply(lambda x: list(x.local_feature_dir)),
        )
        .reset_index()
        .merge(df_filter, on=["file", "index"], how="left",)
        .merge(MAP_INFO[["NextNum", "pos"]], left_on="pacmanPos", right_on="pos")
        .drop(columns=["pos"])
    )
    return df_overlap


def intersect_cnt(df, col1, col2):
    df["intersect_cnt"] = df.apply(
        lambda x: len(set(x[col1]) & set(x[col2]))
        if not isinstance(x[col1], float) and not isinstance(x[col2], float)
        else np.nan,
        1,
    )
    return df


def add_possible_dirs(df_total):
    for w in ["1", "2"]:
        df_total = (
            intersect_cnt(
                df_total.reset_index()
                .merge(
                    POSSIBLE_DIRS, left_on="pacmanPos", right_on="p_choice", how="left"
                )
                .set_index("level_0"),
                "level_1",
                "ghost" + w + "_wrt_pacman",
            )
            .sort_index()
            .rename(columns={"intersect_cnt": "intersect_cnt" + w})
        )
        df_total.loc[
            ~df_total["ghost" + w + "_wrt_pacman"].isnull(), "base" + w
        ] = df_total.loc[
            ~df_total["ghost" + w + "_wrt_pacman"].isnull(), "intersect_cnt" + w
        ] / df_total.loc[
            ~df_total["ghost" + w + "_wrt_pacman"].isnull(), "ghost" + w + "_wrt_pacman"
        ].map(
            len
        )
        df_total = df_total.drop(columns=["p_choice", "level_1", "intersect_cnt" + w])
    return df_total


def toward_ghost_table(df_total, cond=True):
    mapping_d = {"1": "red", "2": "yellow"}
    rs = pd.DataFrame()
    for w in ["1", "2"]:
        if "ghost" + w + "_dimi_manh" not in df_total.columns:
            df_total["ghost" + w + "_dimi_manh"] = (
                df_total[["next_pacman_dir_fill", "ghost" + w + "_wrt_pacman"]]
                .explode("next_pacman_dir_fill")
                .explode("ghost" + w + "_wrt_pacman")
                .apply(
                    lambda x: x["ghost" + w + "_wrt_pacman"]
                    == x["next_pacman_dir_fill"],
                    1,
                )
                .max(level=0)
            )
        rs = pd.concat(
            [
                rs,
                df_total[
                    (df_total.pacmanPos != df_total.pacmanPos.shift(-1))
                    & (df_total["ifscared" + w] != 3)
                    & (df_total["base" + w] == 0.5)
                    & (df_total["distance" + w] > 2)
                    & cond
                    #                     & (
                    #                         #                                                 df_total.index.isin(list(itertools.chain(*select_status[key])))
                    #                         df_total[select_status[key]]
                    #                         == 1
                    #                     )
                ]
                .groupby(
                    [
                        df_total["ifscared" + w] >= 3,
                        df_total["distance" + w].apply(lambda x: min(x, 25)),
                        "pacmanPos",
                        "ghost" + w + "Pos",
                    ]
                )["ghost" + w + "_dimi_manh"]
                .mean()
                .reset_index()
                .drop(columns=["pacmanPos", "ghost" + w + "Pos"])
                .groupby(["ifscared" + w, "distance" + w])["ghost" + w + "_dimi_manh"]
                .agg(["mean", "std", "count"])
                .unstack("ifscared" + w)
                .rename(
                    columns={
                        False: "ghost(" + mapping_d[w] + ") normal",
                        True: "ghost(" + mapping_d[w] + ") scared",
                    }
                ),
            ],
            1,
        )

    rs = rs.stack().reset_index()
    rs = pd.concat(
        [
            rs,
            rs.ifscared1.str.split(" ", expand=True).rename(
                columns={0: "ghost", 1: "status"}
            ),
        ],
        1,
    )
    return rs


def extend_cross_fork(df_overlap, save_path):
    df_overlap = df_overlap.merge(
        LOCS_DF[LOCS_DF.dis == 1].explode("relative_dir"),
        left_on=["pacmanPos", "pacman_dir_fill"],
        right_on=["pos1", "relative_dir"],
        how="left",
    )
    df_overlap = df_overlap[
        (
            (df_overlap.NextNum == 4)
            | ((df_overlap.NextNum == 3) & (df_overlap.pos2.isnull()))
        )
        & (df_overlap.pacman_dir_fill != df_overlap.next_pacman_dir_fill)
    ]
    #     if two_sides_diff:
    df_overlap["left_turn"] = df_overlap.pacman_dir_fill.replace(left_turn_mapping)
    df_overlap["right_turn"] = df_overlap["left_turn"].replace(OPPOSITE_DIRS)
    for d in ["left", "right"]:
        df_temp = pd.concat(
            [
                df_overlap[["up", "down", "right", "left"]].stack().reset_index(1),
                df_overlap[d + "_turn"],
            ],
            1,
        )

        df_overlap = pd.concat(
            [
                df_overlap.drop(columns=[d + "_turn"]),
                df_temp[df_temp.level_1 == df_temp[d + "_turn"]].rename(
                    columns={0: d + "_beans"}
                ),
            ],
            1,
        )
    sns.set_palette(["darkgreen", "khaki"])
    df_overlap.assign(
        ifturn_left=(df_overlap.left_turn == df_overlap.next_pacman_dir_fill).fillna(
            False
        )
    ).groupby(
        [df_overlap.left_beans - df_overlap.right_beans, "NextNum"]
    ).ifturn_left.mean().unstack()[
        -6:4
    ].fillna(
        0
    ).plot(
        marker="o"
    )
    plt.legend(["fork", "cross"])
    plt.ylabel("% of turning left")
    plt.xlabel("# of left beans - right beans")
    plt.ylim(-0.2, 1.2)
    plt.savefig(save_path)


def exclude_2dirs(df_overlap):
    max_2max_columns = select_max_2max_dirs(df_overlap)
    df_overlap = pd.concat([df_overlap, max_2max_columns], 1)

    df_overlap = df_overlap[
        df_overlap.assign(
            pacman_opp=df_overlap.pacman_dir_fill.replace(OPPOSITE_DIRS)
        ).apply(
            lambda x: sorted([x.pacman_opp, x.pacman_dir_fill]) != sorted([x[0], x[1]]),
            1,
        )
    ]
    return df_overlap


def go_to_most_beans(
    df_overlap, cate_df, save_path, only_cross_fork, exclude_2dirs, landscape
):
    pp = []
    for n in [5]:
        if only_cross_fork:
            """这个仅仅适合n=5的情况"""
            extend_cross_fork(df_overlap, save_path)
            break
        if exclude_2dirs:
            df_overlap = exclude_2dirs(df_overlap)

        """1)是否选择了最大的方向choice_large 2) 这个位置是不是转弯口"""
        df_overlap = df_overlap.assign(
            choice_large=df_overlap.apply(
                lambda x: x.next_pacman_dir_fill == random.choice(x.largest_dir), 1
            ),
            if_cross=df_overlap.pacmanPos.isin(TURNING_POS),
        )

        """准备画图元素"""
        """如果需要按照地形来分的话，需要把comment掉的东西都恢复"""
        result_df = (
            df_overlap[df_overlap[["down", "up", "left", "right"]].max(1) > 0]
            .groupby(
                [
                    "NextNum",
                    "if_cross",
                    df_overlap[
                        df_overlap[["down", "up", "left", "right"]].max(1) > 0
                    ].local_4dirs_diff.apply(lambda x: min(x, 4)),
                ]
            )
            .choice_large.apply(
                lambda x: pd.Series({"mean": x.mean(), "count": len(x), "std": x.std()})
            )
            .unstack()
            .reset_index()
        ).merge(cate_df, on=["if_cross", "NextNum"], how="left",)
        #     result_df = result_df[result_df.category.isin(["fork", "cross"])]
        if landscape:
            plt.figure(dpi=300)
            sns.scatterplot(
                data=result_df,
                x="local_4dirs_diff",
                y="mean",
                #             size=result_df["count"],
                ax=ax,
                hue="category",
                hue_order=["straight", "L-shape", "fork", "cross"],
                sizes=(20, 200),
            )
            for c in ["straight", "L-shape", "fork", "cross"]:
                gpd = result_df[result_df.category == c]
                ax.errorbar(
                    gpd["local_4dirs_diff"],
                    gpd["mean"],
                    yerr=gpd["std"] / np.sqrt(gpd["count"]),
                    marker=None,
                    capsize=3,
                )
            ax.legend()
            ax.set_xticks([0, 1, 2, 3, 4])
            ax.set_xticklabels([0, 1, 2, 3, ">=4"])
            ax.set_xlabel("local reward max - 2nd max")
            ax.set_ylabel("% of toward the most valuable direction")
            ax.set_title("errorbar = traditional std")
            ax.set_ylim(0, 1)
            ax.figure.savefig(save_path)
        else:
            errorplot = ax.errorbar(
                result_df[result_df.category == land]["local_4dirs_diff"],
                result_df[result_df.category == land]["mean"],
                #         yerr=result_df["std"] / np.sqrt(result_df["count"]),
                marker="o",
                #         capsize=3,
            )
            pp.append(errorplot.lines[0])

            # use them in the legend
            ax.legend(pp, [str(i) for i in [1, 3, 5, 7]], ncol=4, numpoints=1)
            ax.set_xticks([0, 1, 2, 3, 4])
            ax.set_xticklabels([0, 1, 2, 3, ">=4"])
            plt.xlabel("local reward max - 2nd max")
            plt.ylabel("% of toward the most valuable direction")
            plt.title(land.capitalize() + " Local Graze")
            plt.savefig(save_path)


def add_PEG_dis(df_total):
    diss = add_dis(
        add_dis(
            add_dis(
                df_total[
                    ["ghost2Pos", "ghost1Pos", "next_eat_energizer", "pacmanPos"]
                ].reset_index(),
                "pacmanPos",
                "next_eat_energizer",
                "PE_dis",
            ),
            "next_eat_energizer",
            "ghost1Pos",
            "EG1_dis",
        ),
        "next_eat_energizer",
        "ghost2Pos",
        "EG2_dis",
    )

    diss["EG_dis"] = diss[["EG1_dis", "EG2_dis"]].min(1)

    df_total = pd.concat(
        [
            df_total,
            diss.set_index("level_0")[["PE_dis", "EG_dis", "EG1_dis", "EG2_dis"]],
        ],
        1,
    )
    return df_total


def count_change_dir(x):
    """x: [[(1,2),(4,5)]]"""
    df_temp = pd.DataFrame(x).T.reset_index()
    move_dir = add_move_dir(df_temp, 0, "move_dir", direction=1).move_dir.explode()
    return (move_dir.fillna(method="bfill") != move_dir.fillna(method="bfill").shift())[
        1:
    ].sum()


def generate_cons(df_total):
    cons_list_accident = (
        df_total[df_total.status == "local"]
        .groupby("file")
        .apply(lambda x: x.index.tolist())
        .reset_index()[0]
    )
    cons_list_plan = (
        df_total[df_total.status == "planned_hunting"]
        .groupby("file")
        .apply(lambda x: x.index.tolist())
        .reset_index()[0]
    )
    return cons_list_accident, cons_list_plan


def add_nearest_rwd_reset(df_total):
    """
    对每一步添加最近的点(有可能是多个点，所以对于每个点return的是一个list)
    """
    tmp = add_dis(
        df_total[["beans", "file", "index"]]
        .assign(reset_pos=[(14, 27)] * df_total.shape[0])
        .explode("beans"),
        "reset_pos",
        "beans",
    )
    tmp = (
        tmp.merge(
            tmp.groupby(["file", "index"]).dis.min().reset_index(),
            on=["dis", "file", "index"],
        )
        .groupby(["file", "index", "dis"])
        .apply(lambda x: list(x.beans))
        .reset_index()
        .rename(columns={0: "nearresetPos", "dis": "rwd_reset_distance"})
    )

    return df_total.merge(tmp, on=["file", "index"], how="left")


def label_hunts(df_model):
    hunt_index1 = (
        ((df_model.status_e1 == 0) & (df_model.status_h1 == 1))
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )

    hunt_index2 = (
        ((df_model.status_e2 == 0) & (df_model.status_h2 == 1))
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )
    prehunt_index = (
        ((df_model.status_h1 == -1) | (df_model.status_h2 == -1))
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )
    df_model.loc[hunt_index1, "label_hunt1"] = 1
    df_model.loc[hunt_index2, "label_hunt2"] = 1
    df_model.loc[prehunt_index, "label_prehunt"] = 1


def plot_eating(df_pos, idx, reward_sel):
    f, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(ARRAY, ax=ax, linewidth=0.5, annot=False, cbar=False, cmap="bone")
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    ax.set_title(df_pos.file.values[0])

    plt.scatter(
        reward_sel[(reward_sel.Step == idx) & (reward_sel.Reward <= 2)].X + 0.5,
        reward_sel[(reward_sel.Step == idx) & (reward_sel.Reward <= 2)].Y + 0.5,
        color="brown",
        s=reward_sel[(reward_sel.Step == idx) & (reward_sel.Reward <= 2)].Reward * 70,
        marker=get_marker(SYMBOLS["cookie"]),
    )
    plt.scatter(
        reward_sel[(reward_sel.Step == idx) & (reward_sel.Reward > 2)].X + 0.5,
        reward_sel[(reward_sel.Step == idx) & (reward_sel.Reward > 2)].Y + 0.5,
        color="red",
        s=120,
        marker=get_marker(SYMBOLS["apple"]),
    )
    plt.scatter(
        np.array(df_pos.loc[idx, "pacmanPos"])[0] + 0.5,
        np.array(df_pos.loc[idx, "pacmanPos"])[1] + 0.5,
        marker=get_marker(SYMBOLS["monkey"]),
        s=300,
        color="green",
    )  # pacman

    #     plt.text(
    #         x=40,
    #         y=6,
    #         s="global_Q: " + str(df_pos.loc[idx, "global_Q"]),
    #         horizontalalignment="right",
    #     )
    #     plt.text(
    #         x=40,
    #         y=7,
    #         s="local_Q: " + str(df_pos.loc[idx, "local_Q"]),
    #         horizontalalignment="right",
    #     )
    #     plt.text(
    #         x=40,
    #         y=8,
    #         s="pess_Q: " + str(df_pos.loc[idx, "pessimistic_Q"]),
    #         horizontalalignment="right",
    #     )

    plt.text(
        x=40,
        y=9,
        #         s="current index: " + str(idx + df_pos["index"].values[0]),
        s="current index: " + str(idx),
        horizontalalignment="right",
    )
    plt.text(
        x=40,
        y=10,
        s="handcraft_label: " + ", ".join(np.ravel(df_pos.loc[0, "fitted_label"])),
        horizontalalignment="right",
    )
    plt.text(
        x=40,
        y=11,
        s="estimated_label: " + ", ".join(np.ravel(df_pos.loc[0, "rulebased_label"])),
        horizontalalignment="right",
    )
    for cnt, i in enumerate(
        ["global", "local", "evade(Blinky)", "evade(Clyde)", "approach", "energizer"]
    ):
        plt.text(
            x=40,
            y=12 + cnt,
            s=i + ": " + str(np.round(df_pos.loc[idx, i + "_weight"], 2)),
            horizontalalignment="right",
        )

    if df_pos.loc[idx, "ifscared1"] < 3:
        plt.scatter(
            np.array(df_pos.loc[idx, "ghost1Pos"])[0] + 0.5,
            np.array(df_pos.loc[idx, "ghost1Pos"])[1] + 0.5,
            marker=get_marker(SYMBOLS["ghost"]),
            s=300,
            color="red",
        )  # normal ghost
    elif df_pos.loc[idx, "ifscared1"] >= 4:
        plt.scatter(
            np.array(df_pos.loc[idx, "ghost1Pos"])[0] + 0.5,
            np.array(df_pos.loc[idx, "ghost1Pos"])[1] + 0.5,
            marker=get_marker(SYMBOLS["ghost"]),
            s=300,
            color="white",
            edgecolor="red",
        )  # scared + flashing ghost
    else:
        plt.scatter(
            np.array(df_pos.loc[idx, "ghost1Pos"])[0] + 0.5,
            np.array(df_pos.loc[idx, "ghost1Pos"])[1] + 0.5,
            marker=get_marker(SYMBOLS["eye"]),
            s=30,
            color="white",
            edgecolor="black",  # dead ghost
        )
    if df_pos.loc[idx, "ifscared2"] < 3:
        plt.scatter(
            np.array(df_pos.loc[idx, "ghost2Pos"])[0] + 0.5,
            np.array(df_pos.loc[idx, "ghost2Pos"])[1] + 0.5,
            marker=get_marker(SYMBOLS["ghost"]),
            s=300,
            color="orange",
        )  # normal ghost
    elif df_pos.loc[idx, "ifscared2"] >= 4:
        plt.scatter(
            np.array(df_pos.loc[idx, "ghost2Pos"])[0] + 0.5,
            np.array(df_pos.loc[idx, "ghost2Pos"])[1] + 0.5,
            marker=get_marker(SYMBOLS["ghost"]),
            s=300,
            color="white",
            edgecolor="orange",
        )  # scared + flashing ghost
    else:
        plt.scatter(
            np.array(df_pos.loc[idx, "ghost2Pos"])[0] + 0.5,
            np.array(df_pos.loc[idx, "ghost2Pos"])[1] + 0.5,
            marker=get_marker(SYMBOLS["eye"]),
            s=30,
            color="white",
            edgecolor="black",  # dead ghost
        )


def generate_qtable(df_output, filename, window_size=3):
    Q_table = pd.DataFrame()
    file_data = [j for j in df_output if j[-1] == filename][0]
    set_trace()
    for start_index, sub in enumerate(file_data[4].round(2)):
        df_temp = pd.concat(
            [
                pd.Series([list(i[0]), list(i[1])], index=["local_Q", "pessimistic_Q"],)
                for i in sub
            ],
            1,
        ).T
        df_temp.index = range(start_index, start_index + 2 * window_size + 1)
        Q_table = Q_table.append(df_temp)
    Q_table = (
        Q_table[~Q_table.index.duplicated(keep="first")][window_size:-window_size]
        .reset_index()
        .drop("index", 1)
    )
    return Q_table


def add_pess(x):
    if x.estimated_label == ["pessimistic"]:
        return x.estimated_label
    if x.estimated_label != [""]:
        if x.pessimistic_weight > 0.5:
            return [x.estimated_label[0], "pessimistic"]
        else:
            return x.estimated_label

<IPython.core.display.Javascript object>

#### 全局常量

In [5]:
params = {
    "legend.fontsize": 14,
    "legend.frameon": False,
    "ytick.labelsize": 14,
    "xtick.labelsize": 14,
    "figure.dpi": 300,
    "axes.prop_cycle": plt.cycler("color", plt.cm.tab20b(np.linspace(0, 1, 4))),
    "axes.labelsize": 14,
    "axes.titlesize": 14,
    "pdf.fonttype": 42,
    "font.sans-serif": "Myriad Pro",
    "font.family": "sans-serif",
}
plt.rcParams.update(params)
colormapping = {"1": "firebrick", "2": "khaki"}

"""给result_df加上类别的label"""
cate_df = pd.DataFrame(
    [
        [2, False, "straight"],
        [2, True, "L-shape"],
        [3, True, "fork"],
        [4, True, "cross"],
    ],
    columns=["NextNum", "if_cross", "category"],
)
"""为每个运动方向选择向左的方向"""
left_turn_mapping = {"right": "up", "up": "left", "left": "down", "down": "right"}

sshape = [
    (10, 12),
    (10, 30),
    (13, 12),
    (13, 30),
    (16, 12),
    (16, 30),
    (19, 12),
    (19, 30),
]
delays_up = [(13, 5), (16, 5)]
delays_down = [(7, 30), (22, 30)]
outside = [(2, 5), (2, 24), (2, 12), (2, 33), (27, 5), (27, 24), (27, 12), (27, 33)]

central = [
    i for i in TURNING_POS if i[0] <= 19 and i[0] >= 10 and i[1] >= 15 and i[1] <= 24
]
central = list(set(central) - set([(14, 15), (15, 15)]))

red_pos = [i for i in TURNING_POS if i[0] in [2, 27]] + [
    (7, 24),
    (7, 18),
    (7, 9),
    (22, 24),
    (22, 18),
    (22, 9),
]

blue_pos = list(set(TURNING_POS) - set(red_pos))

state_color = {
    "global graze": plt.cm.Set1.colors[1],
    "local graze in scared mode": plt.cm.Set1.colors[0],
    "local graze in normal mode": plt.cm.Set1.colors[0],
    "hunt": plt.cm.Set1.colors[3],
    "evade": plt.cm.Set1.colors[0],
    "suicide": plt.cm.Set1.colors[1],
    "accidentally hunt": plt.cm.Set1.colors[0],
    "planning hunt": plt.cm.Set1.colors[1],
}

<IPython.core.display.Javascript object>

#### 读取全数据

In [ ]:
df_total_small = pd.read_pickle("../constants/simulated_total.pkl")
with open("../constants/df_overlap_simulated.pkl", "wb") as f:
    pickle.dump(
        generate_simulated_local_4dirs(df_total_small, "beans", joinon="pacmanPos"), f
    )
label_hunts(df_total_small)

cons_list_accident, cons_list_plan = (
    status_index(
        df_total_small[df_total_small.status == "local_on_accidental_three_record"]
    )[0],
    pd.Series(
        status_index(df_total_small[df_total_small.status == "planned_hunting_record"])[
            2
        ]
    ),
)

In [6]:
dall = pickle.load(open("../constants/all_data_new.pkl", "rb"))

locals().update(dall)

# """按照每个game的平均pupil size作为baseline，算出eye_size_std2"""
# df_total = (
#     df_total.reset_index()
#     .merge(
#         df_total.groupby("game")
#         .eye_size.mean()
#         .rename("eye_size_basegame")
#         .reset_index(),
#         on="game",
#         how="left",
#     )
#     .set_index("level_0")
# )
# df_total["eye_size_std2"] = df_total.eye_size - df_total.eye_size_basegame

<IPython.core.display.Javascript object>

In [ ]:
def revise_fruit(x):
    if x.fruitPos.isnull().sum() == x.shape[0]:
        return x
    x.fruitPos = x.fruitPos.fillna(method="ffill").fillna(method="bfill")
    x.beans = x.apply(
        lambda x: list(set(set([x.fruitPos]) | set(x.beans)))
        if not isinstance(x.beans, float)
        else [x.fruitPos],
        1,
    )
    if x[x.pacmanPos == x.fruitPos].shape[0] > 0:
        na_start = x[x.pacmanPos == x.fruitPos].index.values[0] + 1
        if na_start in x.index:
            x.loc[na_start - 1 :, "beans"] = x.apply(
                lambda x: list(set(x.beans) - set([x.fruitPos])), 1
            )
            x.loc[na_start:, "fruitPos"] = np.nan
    return x

df_sub = (
    df_total.assign(inttrial=df_total.game_trial.astype(int))
    .reset_index()
    .sort_values(by=["game", "inttrial", "index"])
    .rename(columns={"level_0": "need_back"})
    .reset_index()
    .drop("level_0", 1)
    .groupby("game")
    .apply(lambda x: revise_fruit(x))
    .set_index("need_back")
    .drop("inttrial", 1)
)

"更新df_total"
dall["df_total"] = df_sub

with open("../constants/all_data_new.pkl", "wb") as f:
    pickle.dump(dall, f)

In [ ]:
# np.save(
#     "/home/qlyang/jiaqi/Pacman-Analysis/Utility_Tree_Analysis/equal_records1.npy",
#     [[file_list[i]] + list(df_output[i]) for i in range(94)],
# )

In [7]:
window_size = 3
window_size_pad = 3
for filename in [
    "equal_records.npy",
]:
    df_output = np.load(
        "/home/qlyang/jiaqi/Pacman-Analysis/Utility_Tree_Analysis/" + filename,
        allow_pickle=True,
    )
    #     df_output = [[i[1:] if type(i) != str else i for i in j] for j in df_output]
    file_list = [f[-1] for f in df_output]
    produced = [i[:-4].split("_")[-1] for i in os.listdir(".")]
    # trial name,
    # global + local + pessimistic + intercept weight,
    # global + local + pessimistic weight*Q scale，
    # estimated labels,
    # hand-crafted labels
    "动态图(测试)"
    for i in list(set(file_list) - set(produced)):
        plt.rcParams["figure.dpi"] = 100
        df_explore = (
            df_total[
                (df_total.file == i.split(".")[0] + ".csv")
                #                 & (
                #                     df_total["index"].between(
                #                         *[j[-1] for j in df_output if j[0] == i][0]
                #                     )
                #                 )
            ]
            .reset_index()
            .drop("level_0", 1)
        )
        cols = [
            "global_weight",
            "local_weight",
            "evade(Blinky)_weight",
            "evade(Clyde)_weight",
            "approach_weight",
            "energizer_weight",
        ]
        if [j[0].shape[0] for j in df_output if j[-1] == i][0] <= 1:
            continue
        df_explore = pd.concat(
            [
                df_explore,
                pd.DataFrame(
                    np.concatenate(
                        [
                            np.zeros([window_size_pad, len(cols)]),
                            [j[1] for j in df_output if j[-1] == i][0],
                            np.zeros([window_size_pad, len(cols)]),
                        ],
                    ),
                    columns=cols,
                ),
                pd.DataFrame(
                    pd.Series(
                        [np.nan] * window_size_pad
                        + [j[2] for j in df_output if j[-1] == i][0]
                        + [np.nan] * window_size_pad
                    )
                ).rename(columns={0: "fitted_label"},),
                pd.DataFrame(
                    pd.Series(
                        [np.nan] * window_size_pad
                        + [j[3] for j in df_output if j[-1] == i][0]
                        + [np.nan] * window_size_pad
                    )
                ).rename(columns={0: "rulebased_label"},),
            ],
            1,
        )

        df_explore.fitted_label = df_explore.fitted_label.apply(
            lambda x: [""] if x is None or isinstance(x, float) else x
        )
        #         df_explore.estimated_label = df_explore.apply(lambda x: add_pess(x), 1,)
        df_explore.rulebased_label = df_explore.rulebased_label.apply(
            lambda x: [""] if x is None or isinstance(x, float) else x
        )
        #         Q_table = generate_qtable(df_output, i, window_size=window_size)
        #         df_explore = pd.concat([df_explore, Q_table], 1)
        reward_sel = (
            pd.concat(
                [
                    df_explore.reset_index()
                    .energizers.explode()
                    .dropna()
                    .apply(lambda x: pd.Series([x[0], x[1]], index=["X", "Y"]))
                    .reset_index()
                    .assign(Reward=2),
                    df_explore.reset_index()
                    .beans.explode()
                    .dropna()
                    .apply(lambda x: pd.Series([x[0], x[1]], index=["X", "Y"]))
                    .reset_index()
                    .assign(Reward=1),
                    df_explore.reset_index()
                    .fruitPos.dropna()
                    .apply(lambda x: pd.Series([x[0], x[1]], index=["X", "Y"]))
                    .reset_index()
                    .assign(Reward=3),
                ]
            )
            .rename(columns={"index": "Step"})
            .sort_values("Step")
        )
        # w = interact(
        #     plot_eating,
        #     df_pos=fixed(df_explore),
        #     reward_sel=fixed(reward_sel),
        #     idx=(
        #         widgets.IntSlider(
        #             min=df_explore.index.min(), max=df_explore.index.max(), step=1, value=3,
        #         )
        #     ),
        # )

        for f in os.listdir(
            "/home/qlyang/jiaqi/Pacman-Analysis/Utility_Tree_Analysis/plot_videos/"
        ):
            if f[0] != ".":
                os.remove(
                    "/home/qlyang/jiaqi/Pacman-Analysis/Utility_Tree_Analysis/plot_videos/"
                    + f
                )
        for idx in range(df_explore.shape[0]):
            plot_eating(df_explore, idx, reward_sel)
            plt.savefig(
                "/home/qlyang/jiaqi/Pacman-Analysis/Utility_Tree_Analysis/plot_videos/"
                + str(idx).zfill(3)
                + ".png",
                bbox_inches="tight",
            )
            plt.close()

        subprocess.call(
            'ffmpeg -framerate 3 -start_number 0 -s 1087x827 -i "/home/qlyang/jiaqi/Pacman-Analysis/Utility_Tree_Analysis/plot_videos/%03d.png" -c:v libx264 -vf "fps=25,scale=1920:1080,format=yuv420p" '
            + filename.split("_")[1]
            + "_"
            + i
            + ".mp4",
            shell=True,
        )
        gc.collect()

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


<IPython.core.display.Javascript object>

In [ ]:
"""
对df_total的数据做修正：
新的filename是all_data_new.pkl，
修改处：
1. label_local_graze, label_local_graze_noghost, label_restl ；
2. fruitPos，部分吃掉了之后该列还有数据，已删除做修正；
3. 添加label_global_ending数据：在global尾部pacman和要吃的bean之间的距离<=5的情况下，label为1，其他是np.nan
"""

def label_status(df_model):
    global_graze = (
        (
            (df_model.status_s != 1)
            & (df_model.status_h1 != 1)
            & (df_model.status_h2 != 1)
            & (df_model.index.isin(global_handler))
        )
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )

    tmp = (
        (
            (
                (
                    (df_model.status_e1 == 1)
                    & (df_model.distance1 <= 10)
                    & (df_model.pac_to_ghost1 != True)
                )
                | (
                    (df_model.status_e2 == 1)
                    & (df_model.distance2 <= 10)
                    & (df_model.pac_to_ghost2 != True)
                )
            )
            & (df_model.status_s != 1)
            & (df_model.status_h1 != 1)
            & (df_model.status_h2 != 1)
        )
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )

    # 只要连续的evade的index
    evade_index = []
    for i in consecutive_groups(tmp):
        a = list(i)
        if len(a) >= 3:
            evade_index.extend(a)

    hunt_index = (
        (
            ((df_model.status_e1 == 0) & (df_model.status_h1 == 1))
            | ((df_model.status_e2 == 0) & (df_model.status_h2 == 1))
        )
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )
    tmp = [list(i) for i in consecutive_groups(hunt_index)]
    planning_index = list(itertools.chain(*[i for i in tmp if len(i) <= 4]))

    hunt_index1 = (
        ((df_model.status_e1 == 0) & (df_model.status_h1 == 1))
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )

    hunt_index2 = (
        ((df_model.status_e2 == 0) & (df_model.status_h2 == 1))
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )

    huntfailure_index = (
        (
            ((df_model.status_e1 == 0) & (df_model.status_h1 == 2))
            | ((df_model.status_e2 == 0) & (df_model.status_h2 == 2))
        )
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )

    graze_nohunt_index = (
        (
            ((df_model.status_e1 == 0) & (df_model.status_h1 == 3))
            & ((df_model.status_e2 == 0) & (df_model.status_h2 == 3))
        )
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )
    # hunt -1 代表prehunt
    prehunt_index = (
        ((df_model.status_h1 == -1) | (df_model.status_h2 == -1))
        .replace({False: np.nan})
        .dropna()
        .index.tolist()
    )
    suicide_index = (
        (df_model.status_s == 1).replace({False: np.nan}).dropna().index.tolist()
    )
    local_graze = list(
        set(
            df_model[
                (df_model.status_g == 1) & (df_model.rwd_pac_distance_new <= 5)
            ].index.values
        )
        - (
            set(global_graze)
            | set(hunt_index)
            | set(prehunt_index)
            | set(suicide_index)
            | set(planning_index)
        )
    )
    local_graze_noghost = df_model[
        (df_model.index.isin(local_graze))
        & (df_model.ifscared1 == 3)
        & (df_model.ifscared2 == 3)
    ].index
    local_graze = list(set(local_graze) - set(local_graze_noghost))

    label_dict = {
        "evade": evade_index,
        "hunt1": hunt_index1,
        "hunt2": hunt_index2,
        "huntfailure_hunt": huntfailure_index,
        "huntfailure_graze": graze_nohunt_index,
        "prehunt": prehunt_index,
        "suicide": suicide_index,
        "local_graze": local_graze,
        "local_graze_noghost": local_graze_noghost,
        "planning": planning_index,
    }

    for k in label_dict.keys():
        df_model.loc[label_dict[k], "label_" + k] = 1
    df_model = df_model.assign(
        label_rest=(
            1 - df_model.filter(regex="label_*").sum(1).apply(lambda x: min(x, 1))
        ).replace({0: np.nan})
    )
    return df_model

df_sub = (
    add_dis(
        df_total.drop(columns=["rwd_pac_distance_new"]).reset_index(),
        "pacmanPos",
        "next_eat_rwd_fill",
        rename="rwd_pac_distance_new",
    )
    .set_index("level_0")
    .loc[df_total.index.values]
)

df_sub = label_status(
    df_sub.drop(
        columns=[
            "label_evade",
            "label_hunt1",
            "label_hunt2",
            "label_huntfailure_hunt",
            "label_huntfailure_graze",
            "label_prehunt",
            "label_suicide",
            "label_local_graze",
            "label_local_graze_noghost",
            "label_planning",
            "label_rest",
        ]
    )
)

"修改fruitPos的内容"
mask = [
    df_sub.loc[i, "fruitPos"] in df_sub.loc[i, "beans"]
    if not isinstance(df_sub.loc[i, "fruitPos"], float)
    and not isinstance(df_sub.loc[i, "beans"], float)
    else False
    for i in range(df_sub.shape[0])
]

df_sub.fruitPos = df_sub.fruitPos.mask(~np.array(mask))

"添加label_global_ending"
df_sub.loc[
    (df_sub.label_global == 1) & (df_sub.rwd_pac_distance_new <= 5),
    "label_global_ending",
] = 1

df_sub.loc[
    [
        i[0]
        for i in [
            list(i)
            for i in consecutive_groups(df_sub.label_global_ending.dropna().index)
        ]
        if len(i) == 1
    ],
    "label_global_ending",
] = np.nan

"更新df_total"
dall["df_total"] = df_sub

with open("../constants/all_data_new.pkl", "wb") as f:
    pickle.dump(dall, f)

In [ ]:
df_total = add_PEG_dis(df_total_small)

df_total.next_pacman_dir_fill = df_total.next_pacman_dir_fill.explode()
df_total.pacman_dir_fill = df_total.pacman_dir_fill.explode()

In [ ]:
# 所有数据
# accident：0.45，plan：0.42
# simulation数据
# accident：0.33，plan：0.61

#### plan hunting和accidentally hunting之间的区别

In [ ]:
mapping = {1: "Accidentally Hunting", 2: "Planned Hunting"}
sns.set_palette(plt.cm.Set1.colors[:2])
for ghost in ["1", "2"]:
    #     for name in ["Omega", "Patamon", "overall"]:
    for name in ["simulated"]:
        i = 1
        df_status = pd.DataFrame()
        for sel_list in [cons_list_accident, cons_list_plan]:
            if name == "Omega":
                sel_list = sel_list[
                    sel_list.explode()
                    .reset_index()
                    .set_index(0)
                    .reset_index()
                    .merge(df_total.reset_index(), left_on=0, right_on="level_0")
                    .groupby("index_x")
                    .apply(
                        lambda x: (x.next_eat_rwd.count() - 1 / len(x) <= 0.2)
                        & (x.level_0.values[-1] <= 686225)
                    )
                ]
            elif name == "Patamon":
                sel_list = sel_list[
                    sel_list.explode()
                    .reset_index()
                    .set_index(0)
                    .reset_index()
                    .merge(df_total.reset_index(), left_on=0, right_on="level_0")
                    .groupby("index_x")
                    .apply(
                        lambda x: (x.next_eat_rwd.count() - 1 / len(x) <= 0.2)
                        & (x.level_0.values[0] > 686225)
                    )
                ]
            else:
                sel_list = sel_list[
                    sel_list.explode()
                    .reset_index()
                    .set_index(0)
                    .reset_index()
                    .merge(df_total.reset_index(), left_on=0, right_on="level_0")
                    .groupby("index_x")
                    .apply(lambda x: ((x.next_eat_rwd.count() - 1) / len(x) <= 0.2))
                ]
            x = df_total.loc[
                (df_total.index.isin(sel_list.apply(lambda x: x[0]).values.tolist()))
                & (df_total.PE_dis > 10),
                ["PE_dis", "EG" + ghost + "_dis"],
            ].sum(1)
            df_status = df_status.append(x.reset_index().assign(cate=mapping[i]))
            print(name, mapping[i], ghost, x.mean(), x.std(), x.shape)
            i += 1
        plt.figure()
        [
            df_status[df_status.cate == i][0].hist(
                bins=20,
                weights=np.ones_like(df_status[df_status.cate == i][0])
                / df_status[df_status.cate == i].shape[0],
                alpha=0.7,
            )
            for i in mapping.values()
        ]
        plt.grid(False)
        plt.tight_layout()
        plt.legend(list(mapping.values()), title=None)
        plt.xlabel("PacMan-Energizer distance + Energizer-Ghost" + ghost + " distance")
        plt.ylabel("probability")
        plt.title(name)
        plt.savefig(
            "../" + name.lower() + "pics/4A" + ghost + "_ghost.pdf",
            bbox_inches="tight",
        )

#### suicide heatmap

In [ ]:
base_cnt = (
    df_total.loc[:, ["game", "game_trial"]]
    .drop_duplicates()
    .merge(
        df_total.groupby("game")["game_trial"].max().reset_index(),
        on="game",
        suffixes=["_curr_trial", "_total_trial"],
    )
).pivot_table(
    columns="game_trial_curr_trial", index="game_trial_total_trial", aggfunc="count"
)

ax = sns.heatmap(
    (
        (
            df_total.loc[df_total.label_suicide == 1, ["game", "game_trial"]]
            .drop_duplicates()
            .merge(
                df_total.groupby("game")["game_trial"].max().reset_index(),
                on="game",
                suffixes=["_curr_trial", "_total_trial"],
            )
        ).pivot_table(
            columns="game_trial_curr_trial",
            index="game_trial_total_trial",
            aggfunc="count",
        )
        / base_cnt
    ).dropna(axis=1, how="all"),
    cmap="binary",
    vmin=0.45,
)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
ax.invert_yaxis()

#### 图1B, 1D

In [ ]:
landscape = True
exclude_2dirs = False
only_cross_fork = False  # 去掉直线的运动方向，是不是向左和向右的比例一致
# two_sides_diff = False  # 只要算左边和右边豆子的区别

name = "simulated"
"""把每个点周围四个方向的豆子数算出来，然后算出1)最大的方向 2)最大的两个方向的豆子的差距 3)添加NextNum 4)pacman的位置"""

landscape_colors = {
    "straight": "firebrick",
    "L-shape": "MediumBlue",
    "fork": "darkgreen",
    "cross": "Khaki",
}

df_sub = df_total_small[df_total_small.status == "local_record"]
for land in landscape_colors.keys():
    set_landcolor(landscape_colors, landscape)
    fig, ax = plt.subplots(dpi=300)
    ## 如果是全部的数据，要换下面这个function
    df_overlap = extend_df_overlap(
        df_sub,
        (
            (df_sub[["distance1", "distance2"]].min(1) >= 3)
            & ((df_sub.ifscared1 >= 4) | (df_sub.ifscared2 >= 4))
        ).tolist(),
        "simulated",
    )
    go_to_most_beans(
        df_overlap,
        cate_df,
        "../" + name + "pics/1D2_over3.pdf",
        only_cross_fork,
        exclude_2dirs,
        landscape,
    )
    if landscape:
        break

#### 图1C

In [ ]:
def plot_1C(df_sub, name):
    df_sub = add_possible_dirs(df_sub)
    rs = toward_ghost_table(df_sub)
    dd = {
        "ghost(red) normal": {"color": "firebrick", "linestyle": "-"},
        "ghost(red) scared": {"color": "SlateBlue", "linestyle": "-",},
        "ghost(yellow) normal": {"color": "khaki", "linestyle": "-"},
        "ghost(yellow) scared": {"color": "Aquamarine", "linestyle": "-",},
    }
    # sns.set_palette(plt.cm.Set1.colors[:2][::-1])
    # key = "2"
    # ghost_color = mapping_d[key]
    f = plt.figure()
    for idx, gpd in rs.groupby(["ifscared1"]):
        plt.errorbar(
            gpd["level_0"],
            gpd["mean"],
            yerr=gpd["std"] / np.sqrt(gpd["count"]),
            marker=None,
            capsize=3,
            barsabove=True,
            label=idx,
            **dd[idx],
        )
    plt.legend()
    plt.xlabel("pacman - ghost distance")
    plt.ylabel("% of toward ghost")
    plt.title("pacman's move diminishes manhat. distance of pacman-ghost",)

    plt.ylim(0, 1)
    f.savefig("../" + name + "pics/1C_test.pdf")

In [ ]:
plot_1C(df_sub, name)

#### 图2B

In [ ]:
def plot_2B(df_total, global_handler_sec):
    df_com = pd.DataFrame()
    for n in [0, -2]:
        initial_index = [i[n] for i in global_handler_sec]
        df_temp = (
            add_dis(df_total.loc[initial_index].explode("beans"), "beans", "pacmanPos")
            .groupby(["file", "index"])
            .apply(lambda x: x[x.dis <= 5].shape[0])
        )
        df_com = pd.concat([df_com, df_temp.reset_index()], 1)

    df_com.columns = ["file", "index", "start", "file", "index", "end"]

    df_com = pd.concat(
        [
            df_com.end.apply(lambda x: int(min(x, 8))).value_counts(normalize=True),
            df_com.start.apply(lambda x: int(min(x, 8))).value_counts(normalize=True),
            (df_com.end - df_com.start)
            .apply(lambda x: int(min(x, 8)))
            .value_counts(normalize=True)
            .rename("end - start"),
        ],
        1,
    )
    sns.set_palette("Set1")
    ax = df_com.loc[0:].plot(kind="bar")
    ax.set_title("bean count at global graze start and end point")
    ax.set_xlabel("# of surrounding beans")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    ax.set_ylabel("normalized trial count")
    ax.figure.savefig("../overallpics/2B.pdf")

plot_2B(df_total, global_handler_sec)

#### 图2C

In [ ]:
def plot_2C(df_total, global_handler):
    df_total.loc[
        ~df_total.assign(beansshift=df_total.beans.shift()).apply(
            lambda x: x.next_eat_rwd in x.beansshift
            if not isinstance(x.beansshift, float)
            else False,
            1,
        ),
        "next_eat_rwd",
    ] = np.nan  # 去掉被吃掉的鬼

    df_result = df_total.groupby("file").apply(if_get_nearbean).reset_index()
    df_result.ifeatnear = df_result.ifeatnear.astype(int)
    df_result = df_result[df_result.start_index.isin(global_handler)]

    df_plot = (
        df_result[df_result.level_1 > 0]
        .groupby(
            ((df_result[df_result.level_1 > 0].rwd_cnt) / 5)
            .map(np.ceil)
            .apply(lambda x: min(8, x))
        )
        .ifeatnear.agg(["mean", "std", "count"])
        .assign(rwd_cnt_actual=np.linspace(5, 40, 8))
    )

    sns.set_palette("tab20b")
    ax = sns.scatterplot(
        size="count", data=df_plot, x="rwd_cnt_actual", y="mean", sizes=(20, 100),
    )
    plt.errorbar(
        data=df_plot,
        x="rwd_cnt_actual",
        y="mean",
        yerr=df_plot["std"] / np.sqrt(df_plot["count"]),
        capsize=3,
    )
    ax.set_xticks(np.linspace(5, 40, 8))
    ax.set_xticklabels(list(np.linspace(5, 40, 8).astype(int))[:-1] + [">=40"])
    plt.ylim(0.5, 1)
    plt.xlabel("# of remaining beans")
    plt.ylabel("% of eating the closet bean")
    plt.savefig("../overallpics/2C.pdf")

In [ ]:
plot_2C(df_total, global_handler)

#### 图2D

In [ ]:
df_sub = df_total[(df_total.status == "global_record")]

actual_shortest = df_sub.groupby("trialid").apply(
    lambda x: pd.Series(
        [
            x[~x.next_eat_rwd.isnull()].index.values[0],
            x.iloc[:2]["pacmanPos"].drop_duplicates(keep="last").index.values[0],
            LOCS_DF[
                (LOCS_DF.pos1 == x.pacmanPos.values[0])
                & (LOCS_DF.pos2 == x.next_eat_rwd.dropna().values[0])
            ].dis.values[0],
            (x.pacman_dir_fill.iloc[1:] != x.pacman_dir_fill.iloc[1:].shift())
            .loc[
                x.iloc[:2]["pacmanPos"]
                .drop_duplicates(keep="last")
                .index.values[0] : x[~x.next_eat_rwd.isnull()]
                .index.values[0]
            ]
            .sum()
            - 1,
            count_change_dir(
                LOCS_DF[
                    (LOCS_DF.pos1 == x.pacmanPos.values[0])
                    & (LOCS_DF.pos2 == x.next_eat_rwd.dropna().values[0])
                ].path.values[0]
            ),
        ],
        index=["curr_index", "prev_index", "dis", "actual_crosses", "shortest_crosses"],
    )
    if x[~x.next_eat_rwd.isnull()].shape[0] > 0
    else pd.Series(
        [np.nan, np.nan, np.nan, np.nan, np.nan],
        index=["curr_index", "prev_index", "dis", "actual_crosses", "shortest_crosses"],
    ),
)

In [ ]:
def plot_2D(actual_shortest, name="overall"):
    ylim = 10
    plt.subplots()
    df_plot = actual_shortest[actual_shortest.dis >= 10]
    data = (
        (df_plot.curr_index - df_plot.prev_index - df_plot.dis)
        .where(lambda x: (x <= 10) & (x >= 0))
        .dropna()
        .value_counts()
        .sort_index()
        .reset_index()
    )
    ax = sns.barplot(
        data=data, x="index", y=data[0] / data[0].sum(), palette="Blues_r",
    )
    ax.set_xticks(range(ylim + 1))
    ax.set_xticklabels(range(ylim + 1))
    ax.set_xlabel("actual trajectory length-shortest trajectory length")
    ax.set_ylabel("normalized trial count")
    ax.figure.savefig("../" + name + "pics/2D-1.pdf")

    plt.subplots()
    data = (
        (df_plot.actual_crosses - df_plot.shortest_crosses)
        .where(lambda x: x >= 0)
        .dropna()
        .apply(lambda x: int(min(x, 5)))
        .value_counts()
        .sort_index()
        .reset_index()
    )
    ax = sns.barplot(data=data, x="index", y=data[0] / data[0].sum(), palette="Blues_r")
    ax.set_xticks(range(6))
    ax.set_xticklabels(list(range(5)) + [">=5"])
    ax.set_xlabel("actual trajectory turns-fewest trajectory turns")
    ax.set_ylabel("normalized trial count")
    ax.figure.savefig("../" + name + "pics/2D-2.pdf")

In [ ]:
plot_2D(actual_shortest, name="simulated")

#### 图2E

In [ ]:
def plot_2E(actual_shortest, name="overall"):
    sns.set_palette("tab20b")
    plot_data = actual_shortest[actual_shortest.dis >= 10]
    for i in [1, 2]:
        plt.subplots()
        if i == 1:
            df_plot = plot_data.groupby(
                plot_data.dis.apply(lambda x: min(x, 35))
            ).apply(
                lambda x: pd.Series(
                    {
                        "avg": (x.curr_index - x.prev_index - x.dis).mean(),
                        "std": (x.curr_index - x.prev_index - x.dis).std()
                        / np.sqrt(x.shape[0]),
                    }
                )
            )
        else:
            df_plot = plot_data.groupby(
                plot_data.dis.apply(lambda x: min(x, 35))
            ).apply(
                lambda x: pd.Series(
                    {
                        "avg": (x.actual_crosses - x.shortest_crosses).mean(),
                        "std": (x.actual_crosses - x.shortest_crosses).std()
                        / np.sqrt(x.shape[0]),
                    }
                )
            )
        plt.errorbar(
            y=df_plot["avg"],
            x=df_plot.index,
            yerr=df_plot["std"],
            marker=None,
            capsize=3,
            barsabove=True,
        )
        # ax.invert_yaxis()
        b, t = plt.ylim()  # discover the values for bottom and top
        plt.ylim(b - 0.5, t + 0.5)  # update the ylim(bottom, top) values
        if i == 1:
            plt.ylabel("actual length - dijkstra distance")
        else:
            plt.ylabel("actual trajectory turns - fewest trajectory turns")
        plt.xlabel("dijkstra distance")
        plt.xticks(
            ticks=range(10, 40, 5), labels=list(range(10, 35, 5)) + [">= 35"],
        )
        plt.savefig("../" + name + "pics/2E-" + str(i) + ".pdf")

In [ ]:
plot_2E(actual_shortest)

#### 图3B

In [ ]:
def plot_3B5C(df_total, states, prefix):
    sns.set_palette(
        [
            "green",
            "firebrick",
            "khaki",
            "PaleTurquoise",
            "red",
            "orange",
            "pink",
            "grey",
        ]
    )
    for state in states:
        fig, ax = plt.subplots(dpi=300)
        if state == "hunt":
            sel_df = df_total[(df_total.label_hunt2 == 1) | (df_total.label_hunt1 == 1)]
        elif state == "global graze":
            sel_df = df_total[df_total.label_global_optimal == 1]
        elif state == "local graze in scared mode":
            sel_df = df_total[
                (df_total.label_local_graze == 1)
                & ((df_total.ifscared1 >= 4) | (df_total.ifscared2 >= 4))
            ]
        elif state == "evade":
            sel_df = df_total[df_total.label_evade == 1]
        elif state == "suicide":
            sel_df = df_total[df_total.label_suicide == 1]
        else:
            sel_df = df_total[
                (df_total.lap'lbel_local_graze == 1)
                & ((df_total.ifscared1 <= 2) & (df_total.ifscared2 <= 2))
            ]
        pd.DataFrame(
            sel_df[
                [
                    "pacman_sacc",
                    "ghost1Pos_sacc",
                    "ghost2Pos_sacc",
                    "beans_sacc",
                    "energizer_sacc",
                    "for_sacc",
                    "back_sacc",
                    "wandering",
                ]
            ]
            .fillna(False)
            .mean()
        ).T.plot(kind="bar", ax=ax)
        ax.set_xticklabels([""], rotation=0)
        ax.set_ylabel("saccade ratio")
        plt.legend(
            [
                "pacman",
                "ghost1",
                "ghost2",
                "beans",
                "energizer",
                "forward",
                "backward",
                "wandering",
            ],
            ncol=2, loc='upper left'
        )
        plt.ylim(0, 0.6)
        plt.title(state)
        plt.savefig("../overallpics/" + prefix + state + ".pdf")

In [ ]:
plot_3B5C(df_total, [
        "hunt",
        "global graze",
        "local graze in scared mode",
        "local graze in normal mode",
],'3B')

#### 图3C

In [ ]:
def plot_3C(df_total, name="overall"):
    sns.set_palette(
        [
            "green",
            "firebrick",
            #      "khaki",
            "orange",
            #      "pink",
            #      "red",
            "PaleTurquoise",
        ]
    )

    upper = 18
    interval = 2
    state_name = {
        "hunt": "hunt",
        "global graze": "global",
        "local graze in scared mode": "local_scared",
        "local graze in normal mode": "local_normal",
    }
    for state in state_name.keys():
        fig, ax = plt.subplots(figsize=(7, 4))
        if state == "hunt":
            sel_index = [
                list(i)
                for i in consecutive_groups(
                    df_total[
                        (df_total.label_hunt2 == 1) | (df_total.label_hunt1 == 1)
                    ].index
                )
            ]
        elif state == "global graze":
            sel_index = [
                list(i)
                for i in consecutive_groups(
                    df_total[(df_total.label_global == 1)].index
                )
            ]
        elif state == "local graze in scared mode":
            sel_index = [
                list(i)
                for i in consecutive_groups(
                    df_total[
                        (df_total.label_local_graze == 1)
                        & ((df_total.ifscared1 >= 4) | (df_total.ifscared2 >= 4))
                    ].index
                )
            ]
        else:
            sel_index = [
                list(i)
                for i in consecutive_groups(
                    df_total[
                        (df_total.label_local_graze == 1)
                        & ((df_total.ifscared1 <= 2) & (df_total.ifscared2 <= 2))
                    ].index
                )
            ]
        for w in [
            "pacman_sacc",
            "ghost",
            "for_sacc",
            "beans_sacc",
        ]:
            if w == "ghost":
                tmp = [
                    len(i)
                    * 25
                    / 60
                    / df_total.loc[i, ["ghost1Pos_sacc", "ghost2Pos_sacc"]].sum(1).sum()
                    for i in sel_index
                    if df_total.loc[i, ["ghost1Pos_sacc", "ghost2Pos_sacc"]]
                    .sum(1)
                    .sum()
                    > 0
                ]
            else:
                tmp = [
                    len(i) * 25 / 60 / df_total.loc[i, w].sum()
                    for i in sel_index
                    if df_total.loc[i, w].sum() > 0
                ]
            tmp = list(filter(lambda x: x <= upper, tmp))
            ax.plot(
                (
                    (
                        pd.cut(
                            pd.Series(tmp).apply(lambda x: min(x, upper)),
                            range(0, upper + interval, interval),
                            labels=list(range(interval, upper + interval, interval)),
                        ).value_counts()
                    ).sort_index()
                    / len(tmp)
                ),
                marker="o",
            )
        plt.legend(
            [
                "pacman_sacc",
                "ghost_sacc",
                "for_sacc",
                #             "back_sacc",
                #             "energizer_sacc",
                "beans_sacc",
            ],
            ncol=2,
        )
        ax.set_xticks(range(int(upper / interval)))
        ax.set_xticklabels(
            list(range(interval, upper + interval, interval))[:-1] + [">=" + str(upper)]
        )
        plt.xlabel("Average Saccade Seconds Interval (bins = " + str(interval) + ")")
        plt.ylabel("Nomarlized Trial Count")
        plt.title(state)
        plt.ylim(0, 1)
        plt.savefig("../" + name + "pics/3C_" + state_name[state] + ".pdf")

In [ ]:
plot_3C(df_total)

#### 图3D

In [ ]:
def read_rt(which_monkey):
    pixel_dfs = pd.read_pickle(
        "../constants/pixel_dfs_" + which_monkey.lower() + ".pkl"
    )
    rt = (
        pd.read_pickle("../constants/rt_with_norm_" + which_monkey.lower() + ".pkl")
        .rename(columns={"pacmanPos_x": "pacmanPos"})
        .reset_index()
        .merge(
            pixel_dfs.pacmanPos.reset_index().rename(
                columns={"index": "level_0_y", "pacmanPos": "first_pos"}
            ),
            on="level_0_y",
            how="left",
        )
        .set_index("level_0_x")
    )

    rt = assign_category(rt)
    rt = pd.concat(
        [
            rt,
            rt[["file", "index"]]
            .assign(
                merge_key=rt.apply(
                    lambda x: list(
                        range(max(int(x["index"]) - 20, 0), int(x["index"]) - 1)
                    ),
                    1,
                )
            )
            .explode("merge_key")
            .reset_index()
            .merge(
                rt[["file", "index"]],
                left_on=["file", "merge_key"],
                right_on=["file", "index"],
            )
            .groupby("level_0_x")
            .size()
            .rename("turns_previous"),
        ],
        1,
    )

    rt["previous_steps"] = rt["index"].diff()  # 这个是进入s弯step之前走的路程
    return rt


# rt = read_rt("omega").append(read_rt("patamon"))

In [ ]:
def plot_3D(rt, area, name_num, conda, condb, name="overall"):
    global state_color
    plt.subplots()
    value_col = "rt_std"
    target_col = "next_eat_rwd_fill"
    tags = ["local graze in normal mode", "global graze"]
    cnt = 0
    stats = add_dis(
        pd.DataFrame(
            [
                [
                    df_total.loc[i[0], "pacmanPos"],
                    df_total.loc[i[-1], "pacmanPos"],
                    len(i) - 1,
                ]
                for i in global_handler_sec
            ],
            columns=["startPos", "endPos", "length"],
        ),
        "startPos",
        "endPos",
    )

    df_rt_local_normal = df_total[
        (df_total.label_local_graze == 1)
        & (df_total.ifscared1 <= 2)
        & (df_total.ifscared2 <= 2)
    ][["file", "index"]]

    df_rt_global = df_total[
        (df_total.label_global_optimal == 1)
        & (
            df_total.index.isin(
                list(
                    itertools.chain(
                        *pd.Series(global_handler_sec)[(stats.length >= 10)].apply(
                            lambda x: x[:-5]
                        )
                    )
                )
            )
        )
    ][["file", "index"]]

    for tag in tags:
        df_ = add_dis(
            rt.loc[
                (rt.category != "fork-side")
                & (rt.pacmanPos.isin(area))
                & (rt.first_pos.isin(TURNING_POS)),
                [
                    "file",
                    "index",
                    value_col,
                    "pacmanPos",
                    "pacman_dir",
                    "level_0_y",
                    "first_pos",
                ],
            ],
            "first_pos",
            "pacmanPos",
        )
        if tag == "local graze in scared mode":
            df_plot = add_dis(
                df_.merge(
                    df_total[
                        (df_total.label_local_graze == 1)
                        & ((df_total.ifscared1 >= 4) | (df_total.ifscared2 >= 4))
                    ][["file", "index", target_col]],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "local graze in normal mode":
            df_plot = add_dis(
                df_.merge(
                    df_total.loc[
                        df_rt_local_normal.index, ["file", "index", target_col]
                    ],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "global graze":
            df_plot = add_dis(
                df_.merge(
                    df_total.loc[df_rt_global.index, ["file", "index", target_col]],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "hunt":
            df_plot = add_dis(
                df_.merge(
                    df_total[(df_total.label_hunt1 == 1) | (df_total.label_hunt2 == 1)][
                        ["file", "index", target_col]
                    ],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "evade":
            df_plot = add_dis(
                df_.merge(
                    df_total[(df_total.label_evade == 1)][
                        ["file", "index", target_col]
                    ],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "suicide":
            df_plot = add_dis(
                df_.merge(
                    df_total[(df_total.label_suicide == 1)][
                        ["file", "index", target_col]
                    ],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "accidentally hunt":
            df_plot = add_dis(
                df_.merge(
                    df_total.loc[
                        df_total.index.isin(list(itertools.chain(*cons_list_accident)))
                    ][["file", "index", target_col]],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        elif tag == "planning hunt":
            df_plot = add_dis(
                df_.merge(
                    df_total.loc[
                        df_total.index.isin(list(itertools.chain(*cons_list_plan)))
                    ][["file", "index", target_col]],
                    on=["file", "index"],
                ).drop(columns="dis"),
                "pacmanPos",
                target_col,
            )
        if value_col == "rt_std":
            df_plot = df_plot[df_plot.dis.fillna(0).between(conda, condb)][value_col]
            df_plot.hist(
                bins=np.linspace(-1.5, 1.5, 31),
                weights=np.ones_like(df_plot) / df_plot.shape[0],
                alpha=0.7,
                color=state_color[tag],
            )
            add_stats(df_plot, color=state_color[tag], x=-1.5, y=0.13 - cnt * 0.1)
        else:
            df_plot = (
                df_plot[df_plot.dis.fillna(0).between(conda, condb)][value_col] / 60
            )
            df_plot.hist(
                bins=np.linspace(-1.5, 3, 46),
                weights=np.ones_like(df_plot) / df_plot.shape[0],
                alpha=0.7,
                color=state_color[tag],
            )
            add_stats(df_plot, color=state_color[tag], x=-1.5, y=0.18 - cnt * 0.1)
        cnt += 1
    plt.ylim(0, 0.25)
    plt.grid(False)
    plt.legend(tags)
    plt.xlabel("RT in seconds")
    plt.ylabel("normalized count")
    # plt.title(value_col)
    plt.savefig("../" + name + "pics/" + name_num + ".pdf")

In [ ]:
plot_3D(rt[rt.file.str.contains("Patamon")], red_pos, "3D1", -1, np.inf, "patamon")

#### 图4A

In [ ]:
def plot_4A(df_total, cons_list_accident, cons_list_plan, name="overall"):
    name_map = {"1": "Blinky", "2": "Clyde"}
    end = 3
    for w in ["1", "2"]:
        bins = range(0, 50, 5)
        f = plt.figure(figsize=(8, 5))
        bb = pd.concat(
            [
                (
                    pd.value_counts(
                        pd.cut(
                            [
                                df_total.loc[i[:end], "distance" + w].mean()
                                for i in cons_list_plan
                            ],
                            bins,
                            labels=range(5, 50, 5),
                        ),
                        normalize=True,
                    )
                    .rename("distance")
                    .reset_index()
                    .assign(category="Planned Attack")
                ),
                (
                    pd.value_counts(
                        pd.cut(
                            [
                                df_total.loc[i[:end], "distance" + w].mean()
                                for i in cons_list_accident
                            ],
                            bins,
                            labels=range(5, 50, 5),
                        ),
                        normalize=True,
                    )
                    .rename("distance")
                    .reset_index()
                    .assign(category="Accidental Attack")
                ),
            ],
        )
        sns.set_palette(plt.cm.Set1.colors[:2][::-1])
        sns.barplot(data=bb, x="index", y="distance", hue="category")

        plt.legend(title=None)
        plt.xlabel("PacMan-Ghost Distance")
        plt.ylabel("Probability")
        plt.ylim(0, 0.4)
        plt.title("Ghost " + name_map[w])
        plt.tight_layout()
        plt.savefig("../" + name + "pics/4Aghost" + w + ".pdf", bbox_inches="tight")

In [ ]:
plot_4A(
    df_total[df_total.file.str.contains("-")],
    cons_list_accident.apply(lambda x: x if x[0] >= -1 else np.nan).dropna().values,
    cons_list_plan.apply(lambda x: x if x[0] >= -1 else np.nan).dropna().values,
)
# from scipy.stats import ttest_ind, ks_2samp

# ks_2samp(
#     [df_total.loc[i[:3], "distance2"].mean() for i in cons_list_accident],
#     [df_total.loc[i[:3], "distance2"].mean() for i in cons_list_plan],
# )

#### 图4B1

In [ ]:
def plot_4B1(df_total, map_indexes_accident, name="overall"):
    df = add_combine_huntdis(df_total, map_indexes_accident)
    """比例"""
    bincut = 2
    df_plot = df[
        (df["rwd_pac_distance"] > 0) & (df.status_h1 <= 1) & (df.status_h2 <= 1)
    ].pivot_table(
        columns=np.ceil(df["rwd_pac_distance"] / bincut) * bincut,
        index=np.ceil(df["combine_dis"] / bincut) * bincut,
        values="combine_hunt",
        aggfunc=lambda x: sum(x == 1) / len(x) if len(x) > 3 else np.nan,
    )
    plt.figure(figsize=(8, 8))
    ax = sns.heatmap(
        df_plot.reindex(range(0, 46, 2)).T.reindex(range(0, 26, 2)).T,
        square=True,
        cmap="coolwarm",
        vmin=0,
        vmax=1,
        cbar_kws={"shrink": 0.7},
    )
    ax.invert_yaxis()
    ax.set_xticklabels(
        [i.get_text().split(".")[0] for i in ax.get_xticklabels()], rotation=0
    )
    ax.set_yticklabels([i.get_text().split(".")[0] for i in ax.get_yticklabels()])
    ax.set_ylabel("P-G Distance")
    ax.set_xlabel("P-Nearest B Distance")
    b, t = plt.ylim()  # discover the values for bottom and top
    plt.ylim(b - 0.5, t + 0.5)  # update the ylim(bottom, top) values
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels([0, 1])
    ax.figure.savefig("../" + name + "pics/4B1.pdf")

In [ ]:
plot_4B1(
    df_total,
    pd.Series(
        [
            list(i)
            for i in consecutive_groups(
                df_total[
                    (df_total.status == "local_record") & (df_total.ifscared1 > 3)
                ].index
            )
        ]
    ),
    "simulated",
)

#### 图4B2

In [ ]:
def plot_4B2(df_total, map_indexes_plan, name="overall"):
    df = df_total.loc[list(itertools.chain(*map_indexes_plan))]

    """结合鬼1和鬼2的hunt"""
    df["combine_hunt"] = (
        df[["label_hunt1", "label_hunt2"]].sum(1).apply(lambda x: min(x, 1))
    )

    df.loc[(df.label_hunt1.isnull()) & (df.label_hunt2 == 1), "combine_dis"] = df.loc[
        (df.label_hunt1.isnull()) & (df.label_hunt2 == 1), "distance2"
    ]
    df.loc[(df.label_hunt1 == 1) & (df.label_hunt2.isnull()), "combine_dis"] = df.loc[
        (df.label_hunt1 == 1) & (df.label_hunt2.isnull()), "distance1"
    ]
    df.loc[
        ((df.label_hunt1 == 1) & (df.label_hunt2 == 1))
        | ((df.label_hunt1.isnull()) & (df.label_hunt2.isnull())),
        "combine_dis",
    ] = df.loc[
        ((df.label_hunt1 == 1) & (df.label_hunt2 == 1))
        | ((df.label_hunt1.isnull()) & (df.label_hunt2.isnull())),
        ["distance2", "distance1"],
    ].min(
        1
    )

    """比例"""
    bincut = 2
    df_plot = df[
        (df["rwd_pac_distance"] > 0) & (df.status_h1 <= 1) & (df.status_h2 <= 1)
    ].pivot_table(
        columns=np.ceil(df["rwd_pac_distance"] / bincut) * bincut,
        index=np.ceil(df["combine_dis"] / bincut) * bincut,
        values="combine_hunt",
        aggfunc=lambda x: sum(x == 1) / len(x) if len(x) > 3 else np.nan,
    )

    plt.figure(figsize=(8, 8))
    ax = sns.heatmap(
        df_plot.reindex(range(0, 38, bincut)).T.reindex(range(0, 30, bincut)).T,
        square=True,
        cmap="coolwarm",
        vmin=0,
        vmax=1,
        cbar_kws={"shrink": 0.7},
    )
    ax.invert_yaxis()
    ax.set_xticklabels(
        [i.get_text().split(".")[0] for i in ax.get_xticklabels()], rotation=0
    )
    ax.set_yticklabels([i.get_text().split(".")[0] for i in ax.get_yticklabels()])
    ax.set_ylabel("P-G Distance")
    ax.set_xlabel("P-Nearest B Distance")
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels([0, 1])
    b, t = plt.ylim()  # discover the values for bottom and top
    plt.ylim(b - 0.5, t + 0.5)  # update the ylim(bottom, top) values
    ax.figure.savefig("../" + name + "pics/4B2.pdf")

In [ ]:
plot_4B2(
    df_total,
    map_indexes_plan
)

#### 图4C

In [ ]:
def plot_4C5C(df_total, mapping, save_name, name="overall"):
    l = []
    for i in [
        "pacman_sacc",
        "ghost1Pos_sacc",
        "ghost2Pos_sacc",
    ]:

        for key, sel_index in mapping.items():
            sacc_per = [df_total.loc[s[-7:], i].sum() / 3 for s in sel_index]
            l.append(
                [
                    i.split("_")[0],
                    np.mean(sacc_per),
                    np.std(sacc_per) / np.sqrt(len(sacc_per)),
                ]
            )

    df_plot = pd.DataFrame(l, columns=["agent", "mean", "std"]).assign(
        status=list(mapping.keys()) * 3
    )
    ax = (
        df_plot.pivot(index="agent", columns="status", values="mean")
        .loc[:, ::-1]
        .plot(
            kind="bar",
            yerr=df_plot.pivot(index="agent", columns="status", values="std").values.T,
            color=plt.cm.Set1.colors[:2][::-1],
        )
    )
    plt.ylabel("Average saccade frequency")
    plt.xlabel("saccade subject")
    plt.legend(title=None)
    ax.set_xticklabels(["Ghost Blinky", "Ghost Clyde", "PacMan Self"], rotation=0)
    plt.savefig("../" + name + "pics/" + save_name + ".pdf", bbox_inches="tight")

In [ ]:
# 686225
plot_4C5C(
    df_total,
    {
        "Planned Attack": cons_list_plan.values,
        "Accidental Attack": cons_list_accident.values,
    },
    "4C",
)

#### 图4C3-4

In [ ]:
def plot_4D(df_total, cons_list_accident, cons_list_plan):
    sns.set_palette(
        [
            "green",
            "firebrick",
            "khaki",
            "PaleTurquoise",
            "red",
            "orange",
            "pink",
            "grey",
        ]
    )
    cnt = 3
    for state in ["accidentally", "planned"]:
        fig, ax = plt.subplots(dpi=300)
        if state == "accidentally":
            sel_df = df_total.loc[
                df_total.index.isin(list(itertools.chain(*cons_list_accident)))
            ]
        else:
            sel_df = df_total.loc[
                df_total.index.isin(list(itertools.chain(*cons_list_plan)))
            ]
        pd.DataFrame(
            sel_df[
                [
                    "pacman_sacc",
                    "ghost1Pos_sacc",
                    "ghost2Pos_sacc",
                    "beans_sacc",
                    "energizer_sacc",
                    "for_sacc",
                    "back_sacc",
                    "wandering",
                ]
            ]
            .fillna(False)
            .mean()
        ).T.plot(kind="bar", ax=ax)
        ax.set_xticklabels([""], rotation=0)
        ax.set_ylabel("saccade ratio")
        plt.legend(
            [
                "pacman",
                "ghost1",
                "ghost2",
                "beans",
                "energizer",
                "forward",
                "backward",
                "wandering",
            ],
            ncol=1
        )
        plt.ylim(0, 0.5)
#         plt.legend(ncol=2)
        plt.title(state)
        plt.savefig("../overallpics/4C" + str(cnt) + ".pdf")
        cnt += 1

In [ ]:
plot_4D(df_total, cons_list_accident, cons_list_plan)

#### 图4D

In [ ]:
def plot_4D(df_total, index_mapping, name):
    if "base1" not in df_total.columns:
        df_total = add_possible_dirs(df_total)

    rs_status = pd.concat(
        [
            toward_ghost_table(
                df_total, df_total.index.isin(list(itertools.chain(*item)))
            ).assign(label=key)
            for key, item in index_mapping.items()
        ]
    )

    rs_status = rs_status[rs_status.status == "normal"]

    mapping_d = {"1": "red", "2": "yellow"}
    sns.set_palette(plt.cm.Set1.colors[:2])
    for key in mapping_d.keys():
        ghost_color = mapping_d[key]
        f = plt.figure()
        for idx, gpd in rs_status[
            rs_status.ghost == "ghost(" + ghost_color + ")"
        ].groupby(["label"]):
            plt.errorbar(
                gpd["level_0"],
                gpd["mean"],
                yerr=gpd["std"] / np.sqrt(gpd["count"]),
                marker=None,
                capsize=3,
                barsabove=True,
                label=idx,
            )
        plt.legend()
        plt.xlabel("pacman - ghost distance")
        plt.ylabel("% of toward ghost")
        plt.title(
            "pacman's move diminishes manhat. distance of pacman-ghost \n("
            + ghost_color
            + " ghost)",
        )
        plt.ylim(0, 1)
        f.savefig("../" + name + "pics/4D" + ghost_color + ".pdf")

In [ ]:
index_mapping = {
    "accidental attack": cons_list_accident,
    "planned attack": cons_list_plan,
}
plot_4D(df_total, index_mapping, "simulated")

#### 图4E

In [ ]:
def rt_before_after_eye(last_index_list, df_total, rt, cutoff, col, cond_col=None):
    after_df, before_df = pd.DataFrame(), pd.DataFrame()
    for i in last_index_list:
        file, index = df_total.loc[i, "file"], df_total.loc[i, "index"]
        before = rt[(rt.file == file) & (rt["index"] < index)].iloc[-cutoff:][
            [col, cond_col]
        ]
        before = before[col].mask(before[cond_col] == 0)
        before = pd.Series(before.values, index=range(before.shape[0], 0, -1))
        before_df = pd.concat([before_df, before], 1)

        after = rt[(rt.file == file) & (rt["index"] > index)].iloc[:cutoff][
            [col, cond_col]
        ]
        after = after[col].mask(after[cond_col] == 0)
        after = pd.Series(after.values, index=range(1, after.shape[0] + 1))
        after_df = pd.concat([after_df, after], 1)
    return after_df, before_df


def plot_4E(
    df_total,
    cons_list_accident,
    map_indexes_accident,
    cons_list_plan,
    map_indexes_plan,
    name="overall",
):
    accident_all = combine_pre_post(cons_list_accident, map_indexes_accident)
    prehunt_all = combine_pre_post(cons_list_plan, map_indexes_plan)

    sns.set_palette(plt.cm.Set1.colors[:2][::-1])
    cutoff = 10
    for compute_list in [prehunt_all, accident_all]:
        after_df, before_df = rt_before_after_eye(
            [
                i[-1]
                for i in compute_list
                if max(df_total.loc[i[-1] + 1, ["ifscared1", "ifscared2"]] == 3)
            ],
            df_total,
            df_total,
            cutoff,
            "eye_size_std2",
            cond_col="eye_size",
        )
        after_sts = pd.DataFrame(
            {
                "mean": after_df.mean(1).values,
                "std": after_df.std(1).values,
                "count": after_df.count(1).values,
            },
            index=range(1, cutoff + 1),
        )
        before_sts = pd.DataFrame(
            {
                "mean": before_df.mean(1).values,
                "std": before_df.std(1).values,
                "count": before_df.count(1).values,
            },
            index=range(-1, -cutoff - 1, -1),
        )
        df_plot = before_sts.append(after_sts).sort_index()
        plt.errorbar(
            df_plot.index,
            df_plot["mean"],
            yerr=df_plot["std"] / np.sqrt(df_plot["count"]),
            marker=None,
            capsize=3,
            barsabove=True,
        )
    #     plt.xticks(df_plot.index[::2], [round(i * 25 / 60, 2) for i in df_plot.index[::2]])
    plt.ylabel("relative pupil size")
    plt.xlabel("seconds before or after eat ghost")
    plt.legend(["planned attack", "accidental attack"])
    plt.savefig("../" + name + "pics/4E.pdf", bbox_inches="tight")

In [ ]:
# 686225
plot_4E(
    df_total[df_total.file.str.contains("-")],
    cons_list_accident.apply(lambda x: x if x[-1] < 10e10 else np.nan).dropna().values,
    map_indexes_accident.apply(lambda x: x if x[-1] < 10e10 else np.nan)
    .dropna()
    .values,
    cons_list_plan.apply(lambda x: x if x[-1] < 10e10 else np.nan).dropna().values,
    map_indexes_plan.apply(lambda x: x if x[-1] < 10e10 else np.nan).dropna().values,
)

#### 图5A1

In [ ]:
def generate_suicide_normal(df_total):
    select_last_num = 100
    suicide_normal = (
        df_total.reset_index()
        .merge(
            (df_total.groupby("file")["label_suicide"].sum() > 0)
            .rename("suicide_trial")
            .reset_index(),
            on="file",
            how="left",
        )
        .sort_values(by="level_0")
        .groupby(["file", "suicide_trial"])
        .apply(lambda x: x.level_0.tail(select_last_num).tolist())
        .reset_index()
    )
    suicide_lists = suicide_normal[suicide_normal["suicide_trial"] == True][0]
    normal_lists = suicide_normal[suicide_normal["suicide_trial"] == False][0]
    return suicide_lists, normal_lists


def plot_5A1(df_total, suicide_lists, normal_lists, name="overall"):
    sns.set_palette(plt.cm.Set1.colors[:2][::-1])
    plt.subplots()
    for compute_list in [suicide_lists, normal_lists]:
        i = 2
        data = pd.Series(
            [
                df_total.loc[j[-i], "eye_size_std2"]
                for j in compute_list
                if i <= len(j) and df_total.loc[j[-i], "eye_size"] != 0
            ]
        )
        data.hist(
            bins=range(-2000, 2000, 100),
            alpha=0.7,
            weights=np.ones(data.shape[0]) / data.shape[0],
        )
        plt.grid(False)
        plt.legend(["suicide", "normal die"])
        plt.xlabel("pupil size - average pupil size per trial")
        plt.ylabel("normalized count")
        plt.xlim(-2000, 2000)
    plt.savefig("../" + name + "pics/5A1histogram.pdf", bbox_inches="tight")

    xaxis = range(10, 1, -1)
    sns.set_palette(plt.cm.Set1.colors[:2][::-1])
    plt.subplots()
    for compute_list in [suicide_lists, normal_lists]:
        data = [
            [
                df_total.loc[j[-i], "eye_size_std2"]
                for j in compute_list
                if i <= len(j) and df_total.loc[j[-i], "eye_size"] != 0
            ]
            for i in xaxis
        ]
        gpd = pd.DataFrame(
            [[np.mean(i), np.std(i), len(i)] for i in data],
            columns=["mean", "std", "count"],
        )
        gpd.index = [round(-(i - 1) * 25 / 60, 2) for i in xaxis]
        plt.errorbar(
            gpd.index,
            gpd["mean"],
            yerr=gpd["std"] / np.sqrt(gpd["count"]),
            marker=None,
            capsize=3,
            barsabove=True,
        )
    plt.legend(["suicide", "normal die"])
    plt.ylabel("relative pupil size")
    plt.xlabel("time before getting caught")
    plt.savefig("../" + name + "pics/5A1line.pdf", bbox_inches="tight")

In [ ]:
suicide_lists, normal_lists = generate_suicide_normal(df_total)

In [ ]:
plot_5A1(
    df_total, suicide_lists, normal_lists,
)

#### 图5A2

In [ ]:
def plot_5A2(df_total, suicide_lists, normal_lists):
    xaxis = range(1, 15)
    sns.set_palette(plt.cm.Set1.colors[:2][::-1])
    if "rt_std" not in df_total.columns:
        df_total = df_total.merge(
            rt[["file", "index", "rt", "rt_std"]], on=["file", "index"], how="left"
        )
    for compute_list in [suicide_lists, normal_lists]:
        data = [
            [
                df_total.loc[j, "rt_std"].dropna().values[-i]
                for j in compute_list
                if df_total.loc[j, "rt_std"].dropna().shape[0] >= i
            ]
            for i in xaxis
        ]
        gpd = pd.DataFrame(
            [[np.mean(i), np.std(i), len(i)] for i in data],
            columns=["mean", "std", "count"],
        )
        gpd.index = [-i for i in xaxis]
        plt.errorbar(
            gpd.index,
            gpd["mean"],
            yerr=gpd["std"] / np.sqrt(gpd["count"]),
            marker=None,
            capsize=3,
            barsabove=True,
        )
    # set_trace()
    plt.legend(["suicide", "normal die"])
    plt.xlabel("# RT to the final point")
    plt.ylabel("average RT")
    plt.savefig("../overallpics/5A2.pdf")

In [ ]:
plot_5A2(df_total, suicide_lists, normal_lists)

#### 图5B

In [ ]:
def plot_5B(df_total, name="overall"):
    plt.tight_layout()
    evade_indexes = (
        df_total[df_total.status == "pessimistic_record"]
        .groupby("file")
        .apply(lambda x: x.index.tolist()[0])
        .values
    )

    df_temp = (
        add_dis(
            df_total.loc[evade_indexes, :].reset_index(),
            "next_eat_rwd_fill",
            "pacmanPos",
        )
        .set_index("level_0")
        .sort_index()
        .dis
    )

    df1 = df_total.loc[evade_indexes, :]
    df1["resetpos"] = [(14, 27)] * df1.shape[0]

    df_temp_re = (
        add_dis(df1.reset_index(), "resetpos", "next_eat_rwd_fill")
        .set_index("level_0")
        .sort_index()
        .dis
    )

    suicide_start_index = (
        df_total[df_total.status == "suicide_record"]
        .groupby("file")
        .apply(lambda x: x.index.tolist()[0])
        .values
    )

    df_com = df_total.loc[suicide_start_index, :].rename(
        columns={"rwd_reset_distance": "reset_dis", "pacman_reset_dis": "suicide_dis"}
    )

    df_hist_suicide = (
        pd.cut(
            df_com.suicide_dis - df_com.reset_dis,
            bins=range(-38, 30, 2),
            labels=range(-36, 30, 2),
        )
        .value_counts(normalize=True)
        .rename("distance")
        .reset_index()
        .assign(category="suicide")
    )
    """
    开始画图
    """
    df_hist_suicide.category = "suicide > 0 ratio: " + str(
        round(df_hist_suicide[df_hist_suicide["index"] > 0].sum().distance, 2)
    )
    df_hist_evade = (
        pd.cut(df_temp - df_temp_re, bins=range(-38, 30, 2), labels=range(-36, 30, 2))
        .value_counts(normalize=True)
        .rename("distance")
        .reset_index()
        .assign(category="evade")
    )
    df_hist_evade.category = "evade > 0 ratio: " + str(
        round(df_hist_evade[df_hist_evade["index"] > 0].sum().distance, 2)
    )

    sns.set_palette(plt.cm.Set1.colors[:2])
    ax = sns.barplot(
        data=pd.concat([df_hist_evade, df_hist_suicide]).sort_values(by="index"),
        x="index",
        y="distance",
        hue="category",
    )
    #     set_trace()
    ax.vlines(
        x=(df_temp - df_temp_re).mean() / 2 + 18,
        ymin=0,
        ymax=0.12,
        linestyle="--",
        color=plt.cm.Set1.colors[0],
    )
    ax.vlines(
        x=(df_com.suicide_dis - df_com.reset_dis).mean() / 2 + 18,
        ymin=0,
        ymax=0.18,
        linestyle="--",
        color=plt.cm.Set1.colors[1],
    )
    plt.legend(title=None)
    plt.ylim(0, 0.2)
    tks, labels = ax.get_xticks(), [i.get_text() for i in ax.get_xticklabels()]
    ax.set_xticks(tks[::4])
    ax.set_xticklabels(labels[::4])
    ax.set_xlabel("(Pacman-Pellet Distance) - (Reset-Pellet Distance)")
    ax.set_ylabel("Probability")
    ax.figure.savefig("../" + name + "pics/5B.pdf", bbox_inches="tight")

In [ ]:
"""
添加reset位置最近的点
添加next_eat_rwd_fill这个字段
"""
df_total_small = add_nearest_rwd_reset(
    df_total_small.reset_index()
    .groupby("file")
    .apply(
        lambda x: x.assign(
            next_eat_rwd_fill=x.next_eat_rwd.fillna(method="bfill")
        ).drop("file", 1)
    )
    .reset_index()
    .drop("level_1", 1)
    .set_index("level_0")
    .sort_index()
)
df_total_small["pacman_reset_dis"] = (
    add_dis(
        df_total_small[["nearresetPos", "pacmanPos"]]
        .explode("nearresetPos")
        .reset_index(),
        "nearresetPos",
        "pacmanPos",
    )
    .sort_values("dis")
    .groupby("index")
    .first()
    .dis
)

In [ ]:
plt.figure(dpi=300)
plot_5B(df_total_small, name="simulated")

#### 图5C

In [ ]:
plot_4C5C(
    df_total[df_total.file.str.contains("Patamon")],
    {
        "Suicide": suicide_lists.apply(lambda x: x if x[0] >= 686225 else np.nan)
        .dropna()
        .values,
        "Normal Die": normal_lists.apply(lambda x: x if x[0] >= 686225 else np.nan)
        .dropna()
        .values,
    },
    "5C",
    "patamon",
)

#### 图5C3-4

In [ ]:
plot_3B5C(df_total, ["evade", "suicide"], '5C')

#### 图5D

In [ ]:
def plot_5d(df_total, name="overall"):
    games = (
        df_total[["game", "game_trial"]]
        .drop_duplicates()
        .groupby("game")
        .count()
        .apply(lambda x: x.game_trial if x.game_trial >= 6 else np.nan, 1)
        .dropna()
        .index.tolist()
    )

    ratios = (
        df_total[df_total.game.isin(games)]
        .groupby("game_trial")
        .apply(lambda x: x.groupby("file").label_suicide.max().fillna(0).mean())
    )

    ratios.index = ratios.index.astype(int)
    ratios.sort_index().plot(marker="o")
    plt.xlabel("trial number")
    plt.ylabel("suicide ratio")
    plt.savefig("../" + name + "pics/5D.pdf", bbox_inches="tight")

In [ ]:
plot_5d(df_total[df_total.file.str.contains("Patamon")], "patamon")